# Variational Quantum Algorithms module

We provide to the user a module for make easier the execution of variational quantum algorithm. We explain in this notebook how to use these functions. Note that, for the moment, the module only handle local executions.

We first import all needed functions from the ``vqa`` module.

In [4]:
from mpqp.execution.vqa import *

In [6]:
from mpqp import QCircuit
from mpqp.gates import *

x, y, z = symbols("x y z")
circuit = QCircuit([Rx(x, 0), Ry(y, 1), Rz(z,0), Rz(z,1), CNOT(0,1)])
print(circuit)

     ┌───────┐┌───────┐     
q_0: ┤ Rx(x) ├┤ Rz(z) ├──■──
     ├───────┤├───────┤┌─┴─┐
q_1: ┤ Ry(y) ├┤ Rz(z) ├┤ X ├
     └───────┘└───────┘└───┘


Once the user defined a circuit depending on variables, using ``symbols``, he can call the function ``minimize`` for two purposes:

 1. Find the minimum energy of a hamiltonian with respect to the state produced by the circuit (VQE)
 2. Minimize a custom and more complicated cost function defined by the user (general VQA)

## Variational Quantum Eigensolver

In [26]:
from mpqp.measures import Observable, ExpectationMeasure
from mpqp.execution import IBMDevice, run

In [2]:
import numpy as np
matrix = np.array([[4,  2,  3, 8],
                   [2, -3,  1, 0],
                   [3,  1, -1, 5],
                   [8,  0,  5, 2]])
hamiltonian = Observable(matrix)

In [7]:
circuit.add(ExpectationMeasure([0, 1], observable=hamiltonian, shots=0))

In [18]:
minimize(circuit, Optimizer.COBYLA, IBMDevice.AER_SIMULATOR, optimizer_options={"maxiter":200})

(-5.062257651301108,
 array([-1.57069179e+00, -3.85656379e-04,  1.69513875e+00]))

## Minimizing a custom cost function

In [45]:
circuit2 = circuit.without_measurements()
def cost_function(params):
    r1 = run(circuit2, IBMDevice.AER_SIMULATOR_STATEVECTOR, {x: params[0], y: params[1], z: params[2]})
    r2 = run(circuit, IBMDevice.AER_SIMULATOR, {x: params[0], y: params[1], z: params[2]})
    return abs(r1.amplitudes[0]) - np.sqrt(r2.expectation_value**3)

In [46]:
minimize(cost_function, Optimizer.COBYLA, nb_params=3, optimizer_options={"maxiter":200})

(-36.044727730788836,
 array([-1.45462257e+00,  2.50036492e-04, -1.57049530e+00]))